In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
import scipy as sp
import sklearn
import random 
import time

In [4]:
train_df = pd.read_csv(r"C:\Users\kgaut\Documents\PROJECTS\titanic\train.csv")

In [5]:
test_df = pd.read_csv(r"C:\Users\kgaut\Documents\PROJECTS\titanic\test.csv")

In [6]:
train_df1 = train_df.copy(deep = True)

In [7]:
data_clean = [train_df1, test_df]

In [8]:
train_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [9]:
train_df1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
print('train columns with null values:\n', train_df1.isnull().sum())

train columns with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [11]:
print('test columns with null values:\n', test_df.isnull().sum())

test columns with null values:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [12]:
train_df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Connaghton, Mr. Michael",male,NaN,NaN,NaN,1601,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [13]:
for dataset in data_clean:
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
    

In [14]:
drop_col = ['PassengerId','Cabin', 'Ticket']
train_df1.drop(drop_col, axis=1, inplace=True)


In [15]:
train_df1.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [16]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [17]:
for dataset in data_clean:
    dataset['FamilySize']=dataset['SibSp']+dataset['Parch']+1
    dataset['IsAlone']=1
    dataset['IsAlone'].loc[dataset['IsAlone']>1]=0
    dataset['Title']=dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
    dataset['FareBin']=pd.qcut(dataset['Fare'],4)
    dataset['AgeBin']=pd.cut(dataset['Age'].astype(int),5)

In [18]:
stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
title_names = (train_df1['Title'].value_counts() < stat_min) #this will create a true false series with title name as index

#apply and lambda functions are quick and dirty code to find and replace with fewer lines of code: https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/
train_df1['Title'] = train_df1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
print(train_df1['Title'].value_counts())

Mr        517
Miss      182
Mrs       125
Master     40
Misc       27
Name: Title, dtype: int64


In [19]:
train_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Name          891 non-null object
Sex           891 non-null object
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked      891 non-null object
FamilySize    891 non-null int64
IsAlone       891 non-null int64
Title         891 non-null object
FareBin       891 non-null category
AgeBin        891 non-null category
dtypes: category(2), float64(2), int64(6), object(4)
memory usage: 85.5+ KB


In [20]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 16 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
FamilySize     418 non-null int64
IsAlone        418 non-null int64
Title          418 non-null object
FareBin        418 non-null category
AgeBin         418 non-null category
dtypes: category(2), float64(2), int64(6), object(6)
memory usage: 46.8+ KB


In [21]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
label=LabelEncoder()

In [22]:
for dataset in data_clean:
    dataset['SexC']= label.fit_transform(dataset['Sex'])
    dataset['EmbarkedC']=label.fit_transform(dataset['Embarked'])
    dataset['TitleC']=label.fit_transform(dataset['Title'])
    dataset['AgeBinC']=label.fit_transform(dataset['AgeBin'])
    dataset['FareBinC']=label.fit_transform(dataset['FareBin'])
    
    Target = ['Survived']
    train_dfx=['Sex', 'Pclass', 'Embarked', 'Title', 'SibSp', 'Parch','Fare','Age','FamilySize', 'IsAlone']
    train_dfcalc=['SexC', 'Pclass', 'EmbarkedC', 'TitleC', 'SibSp', 'Parch','Age','Fare']
    traindf2 = Target+train_dfx
    
    
    print('Original X Y: ', traindf2, '\n')


#define x variables for original w/bin features to remove continuous variables
traindf_x_bin = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code']
traindf_xy_bin = Target + traindf_x_bin
print('Bin X Y: ', traindf_xy_bin, '\n')


#define x and y variables for dummy features original
traindf_dummy = pd.get_dummies(train_df1[train_dfx])
traindf_x_dummy = traindf_dummy.columns.tolist()
traindf_xy_dummy = Target + traindf_x_dummy
print('Dummy X Y: ', traindf_xy_dummy, '\n')

Original X Y:  ['Survived', 'Sex', 'Pclass', 'Embarked', 'Title', 'SibSp', 'Parch', 'Fare', 'Age', 'FamilySize', 'IsAlone'] 

Original X Y:  ['Survived', 'Sex', 'Pclass', 'Embarked', 'Title', 'SibSp', 'Parch', 'Fare', 'Age', 'FamilySize', 'IsAlone'] 

Bin X Y:  ['Survived', 'Sex_Code', 'Pclass', 'Embarked_Code', 'Title_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code'] 

Dummy X Y:  ['Survived', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Age', 'FamilySize', 'IsAlone', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Master', 'Title_Misc', 'Title_Miss', 'Title_Mr', 'Title_Mrs'] 



In [23]:
train1_x, test1_x, train1_y, test1_y = model_selection.train_test_split(train_df1[train_dfcalc], train_df1[Target], random_state=0)

In [24]:
train1_x_bin, test1_x_bin, train1_y_bin, test1_y_bin = model_selection.train_test_split(train_df1[traindf_x_bin], train_df1[Target], random_state=0)

KeyError: "['Sex_Code' 'Embarked_Code' 'Title_Code' 'AgeBin_Code' 'FareBin_Code'] not in index"

In [ ]:
train1_x_dummy, test1_x_dummy, train1_y_dummy, test1_y_dummy = model_selection.train_test_split(traindf_dummy[traindf_x_dummy], train_df1[Target], random_state=0)

In [ ]:
train1_x.shape

In [ ]:
test1_x.shape

In [ ]:
train_df1.shape

In [ ]:
plt.figure(figsize=[16,12])
plt.subplot(231)
plt.boxplot(x=train_df1['Fare'], showmeans=True, meanline=True)
plt.title('Fare Boxplot')
plt.ylabel('fare($)')

plt.subplot(232)
plt.boxplot(x=train_df1['FamilySize'], showmeans=True, meanline=True)
plt.title('familysize Boxplot')
plt.ylabel('familysize#')

plt.subplot(233)
plt.boxplot(x=train_df1['Age'], showmeans=True, meanline=True)
plt.title('Age Boxplot')
plt.ylabel('Age(years)')

plt.subplot(234)
plt.hist(x=[train_df1[train_df1['Survived']==1]['Fare'], train_df1[train_df1['Survived']==0]['Fare']],stacked=True, color=['g','r']
        ,label=['Survived', 'Dead'])
plt.title('fare histogram by survival')
plt.xlabel('fare')
plt.ylabel('# of passengers')
plt.legend()

plt.subplot(235)
plt.hist(x = [train_df1[train_df1['Survived']==1]['Age'], train_df1[train_df1['Survived']==0]['Age']], 
         stacked=True, color = ['g','r'],label = ['Survived','Dead'])
plt.title('Age Histogram by Survival')
plt.xlabel('Age (Years)')
plt.ylabel('# of Passengers')
plt.legend()

plt.subplot(236)
plt.hist(x = [train_df1[train_df1['Survived']==1]['FamilySize'], train_df1[train_df1['Survived']==0]['FamilySize']], 
         stacked=True, color = ['g','r'],label = ['Survived','Dead'])
plt.title('Family Size Histogram by Survival')
plt.xlabel('Family Size (#)')
plt.ylabel('# of Passengers')
plt.legend()



In [ ]:
import seaborn as sns

In [ ]:
fig, saxis = plt.subplots(2,3,figsize=(16,12))
sns.barplot(x='Embarked', y='Survived', data=train_df1, ax=saxis[0,0])
sns.barplot(x='Pclass', y='Survived', order=[1,2,3], data=train_df1, ax = saxis[0,1])
sns.barplot(x='IsAlone', y='Survived', order =[1,0], data=train_df1, ax=saxis[0,2])

sns.pointplot(x='FareBin', y='Survived', data=train_df1, ax=saxis[1,0])
sns.pointplot(x='AgeBin', y='Survived', data=train_df1, ax=saxis[1,1])
sns.pointplot(x='FamilySize', y='Survived', data=train_df1, ax=saxis[1,2])

In [ ]:
fig, (axis1,axis2,axis3)=plt.subplots(1,3, figsize=(14,12))

sns.boxplot(x='Pclass', y='Fare', hue='Survived', data=train_df1, ax=axis1)
axis1.set_title('Pclass vs Fare Survival comparison')

sns.violinplot(x='Pclass', y='Age', hue='Survived', data=train_df1, split=True, ax=axis2)
axis2.set_title('Pclass vs Age Survival comparison')

sns.boxplot(x='Pclass', y='FamilySize', hue='Survived', data=train_df1, ax=axis3)
axis3.set_title('Pclass vs Family Size Survival comparison')


In [ ]:
fig, qaxis = plt.subplots(1,3,figsize=(14,12))

sns.barplot(x='Sex', y='Survived', hue='Embarked', data=train_df1, ax=qaxis[0])
axis1.set_title('Sex vs Embarked Survival comparison')

sns.barplot(x='Sex', y='Survived', hue='Pclass', data=train_df1, ax=qaxis[1])
axis1.set_title('Sex vs Pclass Survival comparison')

sns.barplot(x='Sex', y='Survived', hue='IsAlone', data=train_df1, ax=qaxis[2])
axis1.set_title('Sex vS IsAlone survival comparison') 

In [ ]:
fig, (maxis1, maxis2) = plt.subplots(1,2,figsize=(14,12))

#how does family size factor with sex and survival compare

sns.pointplot(x='FamilySize', y='Survived', hue='Sex', data=train_df1, 
             palette={'male':'blue', 'female':'red'}, markers=['*', 'o'], linestyles=['-','--'], ax=maxis1)


sns.pointplot(x='Pclass', y='Survived', hue='Sex', data=train_df1, 
             palette={'male':'blue', 'female':'red'}, markers=['*', 'o'], linestyles=['-','--'], ax=maxis2)




In [ ]:
# how does embark port facotr with class, sex and survival compare
e= sns.FacetGrid(train_df1, col='Embarked')
e.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', ci=95.0, palette='deep')
e.add_legend()

In [ ]:
a = sns.FacetGrid(train_df1, hue='Survived', aspect=4)
a.map(sns.kdeplot, 'Age', shade=True)
a.set(xlim=(0, train_df1['Age'].max()))
a.add_legend()

In [ ]:
#comparison of sex, class and age by survival
h= sns.FacetGrid(train_df1, row='Sex', col='Pclass', hue='Survived')
h.map(plt.hist, 'Age', alpha=.75)
h.add_legend()

In [ ]:
#pair plot of entire dataset
pp= sns.pairplot(train_df1, hue='Survived', palette='deep', size=1.2, diag_kind='kde', diag_kws=dict(shade=True), 
                plot_kws=dict(s=10))
pp.set(xticklabels=[])

In [ ]:
#correlation heatmap

def correlation_heatmap(df):
    _, ax=plt.subplots(figsize=(14,12))
    colormap=sns.diverging_palette(220,10,as_cmap=True)
    _= sns.heatmap(df.corr(),
                  cmap=colormap, square=True, cbar_kws={'shrink':.9}, ax=ax, annot=True,
                  linewidths=0.1, vmax=1.0, linecolor='white', annot_kws={'fontsize':12})
    plt.title('pearson correlation of features', y=1.05, size=15)
    correlation_heatmap(train_df1)
    